# FLOWUnsteady in Colab

This notebook is an example of how to run [FLOWUnsteady](https://flow.byu.edu/FLOWUnsteady/) simulations on your Google Drive through Colab. It contains instructions for mounting Google Drive into the session, mounting the Julia kernel, and simple examples on how to run FLOWUnsteady and XFOIL. 

Prior to running this notebook, make sure you have created a new directory named `FLOWUnsteady` in your Google Drive (with the path `My Drive/FLOWUnsteady`), and that you have uploaded the files `dotjulia.tar.gz` and `naca0012.csv` into this folder. The folder should look like this:


<img src="https://raw.githubusercontent.com/byuflowlab/FLOWUnsteady/master/docs/resources/colab/colab01.png" alt="Pic here" width="900px">

**NOTE:**  See [flow.byu.edu/FLOWUnsteady/tutorials/colab](https://flow.byu.edu/FLOWUnsteady/tutorials/aeroacoustics/) if you are missing any of these files.

## Mounting Google Drive

First, let's mount the Google Drive while we are still in the Python kernell. This will allow us to transfer files from this session and our Google Drive back and forth.

Upon running it, this cell will ask you to link your Google Drive. Please follow the instructions. Once successfully done, your drive will be mounted under `/content/drive` (you can check that it was successfully mounted under the `Files` tab on the right panel).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Mounting Julia Kernel

This cell installs Julia, associates PyCall with Python 2.7, installs the IJulia kernel for Jupyter, and mounts an image of all FLOW packages needed to run FLOWUnsteady. This process should take about 2 minutes.

In [ ]:
# Installation cell
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    echo '********************** Downloading Julia *****************************'
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
echo '********************** Setting up PyCall *********************************'
julia -e 'using Pkg; pkg"add PyCall;"; ENV["PYTHON"]="/usr/bin/python2"; pkg"build PyCall;"'

echo '********************** Setting up Julia Kernel on Jupyter ****************'
julia -e 'using Pkg; pkg"add IJulia;"'

echo '********************** Installing image of packages **********************'
tar -x --overwrite -f /content/drive/My\ Drive/FLOWUnsteady/dotjulia.tar.gz -C /root/.julia --strip-components 2

echo 'Done'

Now, let's activate the Julia kernel by going to Colab's `Edit` menu and select `Notebook settings` from the drop down. Select *Julia 1.4* as the runtime (set `Hardware accelerator` to `none` since we don't need GPU capabilities).

You should see something like this:

<img src="https://raw.githubusercontent.com/byuflowlab/FLOWUnsteady/master/docs/resources/colab/colab00.png" alt="Pic here" width="400px">

Click on SAVE, and **we are ready to get going.**







Verify that the Julia kernel was successfully installed and is running:

In [ ]:
println("You know we are running Julia if this cell runs sucessfully")

Verify that PyCall was successfully set up with Python 2.7:

In [ ]:
import PyCall
PyCall.pyversion

## AirfoilPrep Example: Run XFOIL

In this example we will use the FLOW package [AirfoilPrep](https://github.com/byuflowlab/AirfoilPrep.jl) to run XFOIL on a NACA 0012 (notice that this can be run on any arbitrary airfoil by simply changing the `airfoil_file` argument to any airfoil contour in your Google Drive)

In [ ]:
import AirfoilPrep
ap = AirfoilPrep

airfoil_file  = "/content/drive/My Drive/FLOWUnsteady/naca0012.csv"  # Airfoil contour to read
Re            = 10e6                       # Reynolds number
Ma            = 0.0                        # Mach number

# Read airfoil contour
x, y = ap.readcontour(airfoil_file; header_len=1, delim=",")

# Generate polar through XFOIL
polar = ap.runXFOIL(x, y, Re; alphas=[i for i in -30:1.0:30],
                              verbose=false, Mach=Ma,
                              iter=100)

# Plot polar
ap.plot(polar; geometry=true, cdpolar=false);

The following cell saves $C_\ell$, $C_d$, and $C_m$ vs angle of attack in a csv file at the indicated location in your Google Drive (go to your Google Drive and verify that the file `naca0012_polar.csv` has been created. You may need to refresh your browser).

In [ ]:
ap.save_polar2(polar, "naca0012_polar.csv"; path="/content/drive/My Drive/FLOWUnsteady/")
println("Done!")

## FLOWUnsteady Example: Tandem Heaving Wing

In this example we run the tandem heaving wing example of FLOWUnsteady that comes with the package. We can't do any visualization in this notebook, but here is a video of what the simulation looks like:

In [ ]:
#@title .
using PyCall
command1 = "<iframe width=\"600\" height=\"337\" src=\"https://www.youtube.com/embed/"
command2 = "?modestbranding=1&autohide=1&showinfo=0&rel=0&controls=1\"
            frameborder=\"0\" allow=\"autoplay; encrypted-media\" 
            allowfullscreen></iframe>"
HTML(command1*"Pch94bKpjrQ"*command2)

HTML{String}("<iframe width=\"600\" height=\"337\" src=\"https://www.youtube.com/embed/Pch94bKpjrQ?modestbranding=1&autohide=1&showinfo=0&rel=0&controls=1\"\n            frameborder=\"0\" allow=\"autoplay; encrypted-media\" \n            allowfullscreen></iframe>")


**NOTE 1:** In order to visualize the simulation, you can move the vtk files under `tandemheaving-example/` (generated by the simulation) out of Colab and into your own computer, and open them with Paraview.

**NOTE 2:** For more information on FLOWUnsteady, check the documentation ([flow.byu.edu/FLOWUnsteady](https://flow.byu.edu/FLOWUnsteady/)).

In [ ]:
import FLOWUnsteady

# Path to examples folder
path_to_examples = joinpath(dirname(pathof(FLOWUnsteady)), "..", "examples")

# Include tandem heaving wing example
include(joinpath(path_to_examples, "tandemheavingwing.jl"))

# Run tandem heaving wing example
tandemheavingwing(; VehicleType=FLOWUnsteady.UVLMVehicle,
                    save_path="tandemheaving-example/");